# Split–apply–combine

One of the most important workflows in data analysis is called **split–apply–combine**:

1. Split the data into groups based on a criterion (e.g., species, marital status).
2. Apply operations to the data within each group.
3. Combine the results from the groups.

Of these, the *apply* step is usually the most complex. In pandas, the *combine* step is always implicitly done for you by the methods we will apply.

## Split

Splitting a data frame into groups is best accomplished using the `groupby` method on it. 

In [12]:
import pandas as pd
countries = pd.DataFrame(
    [ 
        ("Europe","Paris",640.7),
        ("Europe","Amsterdam",41.9),
        ("Africa","Nairobi",580.4),
        ("Africa","Accra",239.6),
        ("Asia","Hanoi",331.7),
        ("Asia","Indonesia",1904.6),
    ],
    index=["France","Netherlands","Kenya","Ghana","Vietnam","Indonesia"], 
    columns=["continent","captial","area"]
)
countries

,continent,captial,area
France,Europe,Paris,640.7
Netherlands,Europe,Amsterdam,41.9
Kenya,Africa,Nairobi,580.4
Ghana,Africa,Accra,239.6
Vietnam,Asia,Hanoi,331.7
Indonesia,Asia,Indonesia,1904.6


In [21]:
bycont = countries.groupby("continent")

Nothing is actually done yet to the data frame. It's just set up for applying operations to each group.

In [22]:
for name,group in bycont:
    print(name)
    print(group)
    print()

Africa
      continent  captial   area
Kenya    Africa  Nairobi  580.4
Ghana    Africa    Accra  239.6

Asia
          continent    captial    area
Vietnam        Asia      Hanoi   331.7
Indonesia      Asia  Indonesia  1904.6

Europe
            continent    captial   area
France         Europe      Paris  640.7
Netherlands    Europe  Amsterdam   41.9



You can group by ranges of a quantitative value using `cut`.

In [14]:
cuts = pd.cut(countries["area"],[0,300,600,900,2000])
bysize = countries.groupby(cuts)
for name,group in bysize:
    print(name)
    print(group)
    print()

(0, 300]
            continent    captial   area
Netherlands    Europe  Amsterdam   41.9
Ghana          Africa      Accra  239.6

(300, 600]
        continent  captial   area
Kenya      Africa  Nairobi  580.4
Vietnam      Asia    Hanoi  331.7

(600, 900]
       continent captial   area
France    Europe   Paris  640.7

(900, 2000]
          continent    captial    area
Indonesia      Asia  Indonesia  1904.6



## Apply

The most complex step is applying operations to each group of data. There are three types of operations:

* **Aggregation** refers to summarizing data by a single value, such as a sum or mean, or by a few values, such as value counts or quintiles.
* **Transformation** refers to application of a mathematical operation to every data value, resulting in data indexed the same way as the original. For example, quantitative data might be transformed to lie in the interval $[0,1]$.
* **Filtration** refers to inclusion/removal of a group based on a criterion, such as rejection of a group with too few members.

### Aggregation

Many common operations are defined for aggregation.

In [15]:
bysize.sum()

,area
area,
"(0, 300]",281.5
"(300, 600]",912.1
"(600, 900]",640.7
"(900, 2000]",1904.6


In [16]:
bysize.count()

,continent,captial,area
area,,,
"(0, 300]",2,2,2
"(300, 600]",2,2,2
"(600, 900]",1,1,1
"(900, 2000]",1,1,1


In [17]:
bysize["continent"].value_counts()

area         continent
(0, 300]     Africa       1
             Europe       1
(300, 600]   Africa       1
             Asia         1
(600, 900]   Europe       1
(900, 2000]  Asia         1
Name: continent, dtype: int64

A list of the most common predefined aggregation functions is given in {numref}`table-aggregators`. These functions ignore `NaN` (missing) values. 

```{list-table} Aggregation functions
:name: table-aggregators
* - `mean`
  - Mean of group values
* - `sum`
  - Sum of group values
* - `count`
  - Count of group values
* - `std`, `var`
  - Standard deviation or variance within groups
* - `describe`
  - Descriptive statistics
* - `first`, `last`
  - First or last of group values
* - `min`, `max`
  - Min or max within groups
```


If you want a more exotic operation, you can call `agg` with your own function.

In [30]:
def weirdo(x):
    return sum((i+1)*xi for (i,xi) in enumerate(x))
bycont["area"].agg(weirdo)


continent
Africa    1059.6
Asia      4140.9
Europe     724.5
Name: area, dtype: float64

You may want to import the `numpy` package for common mathematical functions.

In [31]:
import numpy as np
def rms(x):
    return np.sqrt(sum(xi**2 for xi in x))
bycont["area"].agg(rms)


continent
Africa     627.911077
Asia      1933.268230
Europe     642.068610
Name: area, dtype: float64

### Transformation

In the simplest case, a transformation applies a function to each element of a column, producing a result of the same length that can be indexed the same way.

Here is some data where the columns have different scales. 

In [53]:
import numpy as np
rand = np.random.uniform

df = pd.DataFrame({"small": rand(0,0.004,8), "big": rand(100,200,8)})
df

,small,big
0,0.000403,118.980301
1,0.003294,159.832257
2,0.003214,137.174631
3,0.002976,186.948115
4,0.000501,133.836924
5,0.001337,114.810663
6,0.000424,180.570842
7,0.002886,154.775507


Suppose that we want to standardize each column to the interval $[0,1]$. If $m$ and $M$ are the minimum and maximum values in a column, then we can apply the function

$$f(x) = \frac{x-m}{M-m}$$

within that column.

In [54]:
def scale(x):
    m = x.min()
    M = x.max()
    return (x-m)/(M-m)

df.transform(scale)

,small,big
0,0.000000,0.057801
1,1.000000,0.624108
2,0.972398,0.310019
3,0.889783,1.000000
4,0.033958,0.263750
5,0.322991,0.000000
6,0.007198,0.911596
7,0.858697,0.554010


Notice how each column of the result has entries equal to 0 and 1, with all the other values in-between.

Transformations are one mechanism for replacing missing values. Let's modify the original data frame to get some `NaN` entries (provided by `np.nan`).

In [57]:
df.loc[3,"small"] = np.nan
df.loc[6,"small"] = np.nan
df.loc[1,"big"] = np.nan
df

,small,big
0,0.000403,118.980301
1,0.003294,NaN
2,0.003214,137.174631
3,NaN,186.948115
4,0.000501,133.836924
5,0.001337,114.810663
6,NaN,180.570842
7,0.002886,154.775507


Now we want to replace each missing value with the average for its column. We use `fillna` to do the replacements and an aggregator to compute the mean.

In [59]:
def replace(col):
    return col.fillna(col.mean())

df.transform(replace)

,small,big
0,0.000403,118.980301
1,0.003294,146.728140
2,0.003214,137.174631
3,0.001939,186.948115
4,0.000501,133.836924
5,0.001337,114.810663
6,0.001939,180.570842
7,0.002886,154.775507


When a transformation is applied to a grouped data frame, then the transformation is applied to each group. For example, let's consider our rescaling example above, but for values that are indexed by time.

In [60]:
import numpy as np
rand = np.random.uniform

idx = pd.date_range("1/1/2021","12/31/2021",freq="D")
df = pd.DataFrame({"small": rand(0,0.004,365), "big": rand(100,200,365)},index=idx)
df

,small,big
2021-01-01,0.002803,170.854312
2021-01-02,0.001629,163.283119
2021-01-03,0.002434,188.861869
2021-01-04,0.000589,170.809689
2021-01-05,0.000918,103.974490
...,...,...
2021-12-27,0.003071,131.202242
2021-12-28,0.002766,199.070811
2021-12-29,0.000688,176.769428
2021-12-30,0.003341,183.478962


Here is how the data can be grouped by month. (Any timestamp value has properties such as `day` and `month` that return the appropriate value.)

In [61]:
bymonth = df.groupby((lambda x: x.month))
print(len(bymonth))

12


Now, when we transform using our `scale` function from above, each column is rescaled independently within each month.

In [63]:
dfs = bymonth.transform(scale)
dfs

,small,big
2021-01-01,0.699387,0.714391
2021-01-02,0.401880,0.637095
2021-01-03,0.605756,0.898235
2021-01-04,0.138259,0.713935
2021-01-05,0.221519,0.031597
...,...,...
2021-12-27,0.790287,0.312430
2021-12-28,0.707720,1.000000
2021-12-29,0.145431,0.774067
2021-12-30,0.863376,0.842040


For instance, each month has a 0 and a 1 in each column.

In [70]:
dfs.loc[dfs["small"]==1.0]

,small,big
2021-01-29,1.0,0.651895
2021-02-14,1.0,0.135380
2021-03-11,1.0,0.289211
2021-04-25,1.0,0.967453
2021-05-04,1.0,1.000000
2021-06-03,1.0,0.877189
2021-07-24,1.0,0.791884
2021-08-05,1.0,0.413734
2021-09-12,1.0,0.640728
2021-10-04,1.0,0.909632


## Filtering

To apply a filter, provide a function that operates on a column and returns either `True`, meaning to keep the column, or `False`, meaning to reject it. For example, with our time series data above, suppose we want to keep the data only in those groups (months) for which the average value in the `big` column is at least 150.

In [75]:
def isbig150(df):
    return df["big"].mean() > 150

selected = bymonth.filter(isbig150)
print(len(selected))

154


The result of the `filter` operation is a new data frame. If we again group by month and aggregate the mean value in each month, we can verify that the filtering property was satisfied.

In [78]:
selected.groupby(lambda x: x.month).mean()

,small,big
3,0.002180,151.629453
6,0.002663,155.514867
8,0.001928,156.122592
10,0.002101,154.802362
12,0.002002,156.493268
